In [6]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
from torchvision.transforms import Compose
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms as trans

Ниже представлена реализация Cyrcle gan на python с использованием библиотеки torch.

In [2]:
class Convblock(nn.Module):
    def __init__(self,in_chennels,out_chennels, stride = 2, down = True, use_act = True, **kwargs):
        super().__init__()
        
        if down:
            conv = nn.Conv2d(in_chennels, out_chennels, kernel_size=3, stride = stride, padding = 1,padding_mode='reflect',**kwargs)
            
        else:
            conv = nn.ConvTranspose2d(in_chennels, out_chennels, kernel_size=3, stride = stride, padding = 1,**kwargs)
            
        norm = nn.InstanceNorm2d(out_chennels)
        if use_act:
            act = nn.ReLU(inplace=True)
        else:
            act = nn.Identity()
        
        self.conv = nn.Sequential(conv,norm,act)
    
    def forward(self,x):
        return self.conv(x)

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, chennels):
        super().__init__()
        
        self.res = nn.Sequential(
            Convblock(chennels,chennels,stride=1))
        
        self.res = nn.Sequential(
            Convblock(chennels,chennels,stride=1,use_act=False))
    def forward(self, x):
        return x + self.res(x)
        

In [4]:
class Generator(nn.Module):
    def __init__(self,in_chenels = 3 ,out_channels = 64, num_resudials = 9):
        super().__init__()
        
        self.initial = nn.Sequential(            
            nn.Conv2d(in_chenels, out_channels, kernel_size=7, stride = 1, padding = 3),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True))
        
        self.down = nn.Sequential(Convblock(out_channels, out_channels * 2),
                                  Convblock(out_channels * 2, out_channels * 4)
                                 )
        
        self.res = nn.Sequential(*[ResidualBlock(out_channels * 4) for _ in range(num_resudials)])
        
        self.up = nn.Sequential(Convblock(out_channels * 4, out_channels * 2,down = False, output_padding = 1),
                               Convblock(out_channels * 2, out_channels, down = False, output_padding = 1))
        
        self.last = nn.Conv2d(out_channels,in_chenels, kernel_size=7, stride=1, padding = 3, padding_mode='reflect')
    
    def forward(self,x):
        x = self.initial(x)
        x = self.down(x)
        x = self.res(x)
        x = self.up(x)
        x = self.last(x)
        
        return torch.tanh(x) 

In [5]:
class Convblock(nn.Module):
    def __init__(self,in_chennels,out_chennels, stride = 2, down = True, use_act = True, **kwargs):
        super().__init__()
        
        if down:
            conv = nn.Conv2d(in_chennels, out_chennels, kernel_size=3, stride = stride, padding = 1,padding_mode='reflect',**kwargs)
            
        else:
            conv = nn.ConvTranspose2d(in_chennels, out_chennels, kernel_size=3, stride = stride, padding = 1,**kwargs)
            
        norm = nn.InstanceNorm2d(out_chennels)
        if use_act:
            act = nn.ReLU(inplace=True)
        else:
            act = nn.Identity()
        
        self.conv = nn.Sequential(conv,norm,act)
    
    def forward(self,x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, chennels):
        super().__init__()
        
        self.res = nn.Sequential(
            Convblock(chennels,chennels,stride=1))
        
        self.res = nn.Sequential(
            Convblock(chennels,chennels,stride=1,use_act=False))
    def forward(self, x):
        return x + self.res(x)
        

class Generator(nn.Module):
    def __init__(self,in_chenels = 3 ,out_channels = 64, num_resudials = 9):
        super().__init__()
        
        self.initial = nn.Sequential(            
            nn.Conv2d(in_chenels, out_channels, kernel_size=7, stride = 1, padding = 3),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True))
        
        self.down = nn.Sequential(Convblock(out_channels, out_channels * 2),
                                  Convblock(out_channels * 2, out_channels * 4)
                                 )
        
        self.res = nn.Sequential(*[ResidualBlock(out_channels * 4) for _ in range(num_resudials)])
        
        self.up = nn.Sequential(Convblock(out_channels * 4, out_channels * 2,down = False, output_padding = 1),
                               Convblock(out_channels * 2, out_channels, down = False, output_padding = 1))
        
        self.last = nn.Conv2d(out_channels,in_chenels, kernel_size=7, stride=1, padding = 3, padding_mode='reflect')
    
    def forward(self,x):
        x = self.initial(x)
        x = self.down(x)
        x = self.res(x)
        x = self.up(x)
        x = self.last(x)
        
        return torch.tanh(x) 